In [1]:
import os
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
from IPython.display import Image
data_type = 'float32'
os.environ['THEANO_FLAGS'] = 'device=cuda,floatX=' + data_type + ',force_device=True' + ',dnn.enabled=False'
import cell2location

## This model is required for WTA data
os.chdir('../code/')
import spacejam

Can not use cuDNN on context None: Disabled by dnn.enabled flag
ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "/software/team283/tl7/miniconda3/envs/spacejam/lib/python3.7/site-packages/theano/gpuarray/__init__.py", line 227, in <module>
    use(config.device)
  File "/software/team283/tl7/miniconda3/envs/spacejam/lib/python3.7/site-packages/theano/gpuarray/__init__.py", line 214, in use
    init_dev(device, preallocate=preallocate)
  File "/software/team283/tl7/miniconda3/envs/spacejam/lib/python3.7/site-packages/theano/gpuarray/__init__.py", line 159, in init_dev
    pygpu.blas.gemm(0, tmp, tmp, 0, tmp, overwrite_c=True)
  File "pygpu/blas.pyx", line 149, in pygpu.blas.gemm
  File "pygpu/blas.pyx", line 47, in pygpu.blas.pygpu_blas_rgemm
pygpu.gpuarray.GpuArrayException: (b'nvrtcCompileProgram: NVRTC_ERROR_INVALID_OPTION', 3)


In [2]:
from spacejam.models import LocationModelWTAMultiExperimentHierarchicalGeneLevel

models = [spacejam.models.LocationModelWTAMultiExperimentHierarchicalGeneLevel.LocationModelWTAMultiExperimentHierarchicalGeneLevel]

## Reload expression model

In [3]:
inf_aver=pd.read_csv('../data/C2L_RegressionModel_Mean_Expression_sc_ref_covid19.csv', sep=',', index_col=0)
inf_aver

,B.Cell,B.Plasma.IgA,B.Plasma.IgG,EC.Aerocyte,EC.Arterial,EC.Capillary,EC.Lymphatic,EC.Venous.Pul,EC.Venous.Sys,EP.AT1,...,Macro.IV,Monocyte,Mast,Mesothelial,NK,Pericytes,Proliferating.T.NK,T.CD4+,T.CD8+,Vascular.SMC
SAMD11,0.001287,0.001156,0.002704,0.000210,0.001495,0.000427,0.000773,0.058121,0.013561,0.000754,...,0.000087,0.000115,0.000471,0.023930,0.000418,0.001700,0.000518,0.000439,0.000170,0.010850
NOC2L,0.014256,0.035807,0.042609,0.012599,0.025862,0.012190,0.035682,0.038615,0.047270,0.041312,...,0.021555,0.008065,0.003250,0.021676,0.008666,0.019360,0.029805,0.016387,0.009387,0.022757
HES4,0.004630,0.000657,0.000093,0.015595,0.125246,0.027008,0.083245,0.010178,0.033947,0.159227,...,0.012642,0.017076,0.007478,0.006705,0.000733,0.262867,0.001549,0.006740,0.000547,0.390176
ISG15,0.002005,0.003593,0.000526,0.007281,0.014567,0.002677,0.012852,0.036778,0.033474,0.028151,...,0.034168,0.012192,0.002110,0.023320,0.001918,0.021250,0.008253,0.002269,0.000930,0.035484
AGRN,0.001757,0.000723,0.000301,0.019708,0.063464,0.018235,0.035212,0.071161,0.048634,0.287614,...,0.009651,0.005047,0.004579,0.059185,0.000180,0.031603,0.000395,0.002282,0.000183,0.019445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC002072.1,0.004171,0.029677,0.009298,0.000143,0.000302,0.000051,0.005060,0.000220,0.000711,0.000089,...,0.000039,0.000067,0.000680,0.165198,0.000221,0.000362,0.000727,0.000167,0.000126,0.000353
SEPTIN6,0.033502,0.004899,0.000126,0.000120,0.000226,0.000051,0.000599,0.000212,0.000539,0.000068,...,0.000049,0.000069,0.000535,0.007058,0.000280,0.000326,0.041903,0.000766,0.000302,0.000302
AL008633.1,0.001268,0.000233,0.000115,0.009603,0.015359,0.007700,0.057451,0.012550,0.008484,0.182776,...,0.000038,0.000165,0.000410,0.018332,0.000136,0.017341,0.000267,0.000081,0.000077,0.020226
IGF2.1,0.001546,0.000429,0.000121,0.000146,0.001484,0.000080,0.000485,0.001149,0.001523,0.000087,...,0.000038,0.000053,0.000402,0.027912,0.000176,0.000302,0.000487,0.000098,0.000096,0.000388


### Prepare COVID-19 data

In [4]:
adata = sc.read_h5ad("../data/AnnData_WTA_COVID-19.h5ad")

adata_neg = sc.read_h5ad("../data/AnnData_WTA_COVID-19_NegProbes.h5ad")

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


### Run Cell2Location

In [ ]:
for img_patho in ['Normal-like', 'Altered']:
    nuclei_number=np.reshape(np.array(adata[adata.obs['Image_Pathology'] == img_patho].obs['Nuclei'], dtype='float32'), (len(adata[adata.obs['Image_Pathology'] == img_patho].obs['Nuclei']),1))
    for m in [models[0]]:
        cell2location.run_c2l.run_cell2location(
            inf_aver, 
            adata[adata.obs['Image_Pathology'] == img_patho], 
            ignore_deprecation_exception=True,
            model_name=m,
            export_args={'path': '../data/',
                         'run_name_suffix': f'_COVID19_{img_patho}'},
            train_args={'use_raw': True, 'n_iter': 20000, 'sample_name_col': 'Donor_AltName'},
            model_kwargs={
                "Y_data" : adata_neg[adata.obs['Image_Pathology'] == img_patho].X,
                "gene_level_prior": {'mean': 1 / 2, 'sd': 1 / 4, 'sample_alpha': 20},
                "cell_number_prior" : {'cells_per_spot': nuclei_number, 
                                       'factors_per_spot': 7, 'combs_per_spot': 7},
                "cell_number_var_prior" : {'cells_mean_var_ratio': 1e6, 
                                           'factors_mean_var_ratio': 100, 'combs_mean_var_ratio': 1}})